In [38]:
import pandas as pd
import numpy as np
import string
import re
import requests
from bs4 import BeautifulSoup

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
import wikipedia

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
# https://levelup.gitconnected.com/two-simple-ways-to-scrape-text-from-wikipedia-in-python-9ce07426579b

my_url = 'https://en.wikipedia.org/wiki/NBA'
r = requests.get(my_url)
soup = BeautifulSoup(r.text, 'html.parser')

# what elements are in soup 
#print(set([text.parent.name for text in soup.find_all(text=True)]))

# Extract the plain text content from paragraphs
paras = []
for paragraph in soup.find_all('p'):
    paras.append(str(paragraph.text))

# Extract text from paragraph headers
heads = []
for head in soup.find_all('span', attrs={'mw-headline'}):
    heads.append(str(head.text))

# Interleave paragraphs & headers
my_text = [val for pair in zip(paras, heads) for val in pair]
my_text = ' '.join(my_text)

# Drop footnote superscripts in brackets
my_text = re.sub(r"\[.*?\]+", '', my_text)

# Replace '\n' (a new line) with '' and end the string at $1000.
my_text = my_text.replace('\n', '')[:-11]

# NLTK
bw = nltk.word_tokenize(my_text)

def decontracted(text):
    '''convert contractions with apostrophs into words'''
    
    # specific
    phrase = re.sub(r"won\'t", "will not", text)
    phrase = re.sub(r"can\'t", "can not", text)

    # general
    phrase = re.sub(r"n\'t", " not", text)
    phrase = re.sub(r"\'re", " are", text)
    phrase = re.sub(r"\'s", " is", text)
    phrase = re.sub(r"\'d", " would", text)
    phrase = re.sub(r"\'ll", " will", text)
    phrase = re.sub(r"\'t", " not", text)
    phrase = re.sub(r"\'ve", " have", text)
    phrase = re.sub(r"\'m", " am", text)
    
    return text

stop = set(stopwords.words('english') + list(string.punctuation))

#remove stop words, punctuation, numbers, uppercase
bw = [decontracted(i) for i in word_tokenize(my_text.lower()) if not i in stop and i.isalpha()]

lemmatizer = WordNetLemmatizer()
bw = [lemmatizer.lemmatize(i) for i in bw] # lemmatization for nouns
bw = [lemmatizer.lemmatize(i, pos='a') for i in bw] # lemmatization for adjectives

df = pd.DataFrame(bw, columns=['word']).groupby('word').size().sort_values(ascending=False)
df.head(20)

word
nba             52
league          50
season          27
team            19
basketball      19
championship    18
player          15
final           15
first           14
lakers          14
franchise       13
new             13
celtic          12
spur            10
led             10
title           10
warrior         10
game            10
four            10
baa             10
dtype: int64